Выгрузка всех вакансий и запись их в БД
---

In [2]:
%%time
from data_parsers.hhParser import HhParser
from services.db_service import DbService

parser = HhParser()
db_service = DbService()

vacancies = parser.get_vacancies()
db_service.save_vacancies(vacancies)

Получение id вакансий:   0%|          | 0/19 [00:00<?, ?it/s]

Выгрузка вакансий:   0%|          | 0/2000 [00:00<?, ?it/s]

Сохранение вакансий в БД:   0%|          | 0/2000 [00:00<?, ?it/s]

----> Into table vacancies_id was inserted 2000 vacancies
Wall time: 11min 44s
